# Setup Libraries and Enviroment Variables

In [2]:
# Load necessary libraries and Account IDs
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

database_name = "insta_db"

from pyathena import connect

In [3]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [4]:
# Make Connections
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

# Organize Data for Logistic Model

In [ ]:
# SQL statement
statement = """
DROP TABLE IF EXISTS insta_db.final_parquet
"""
import pandas as pd

pd.read_sql(statement, conn)


In [ ]:
# SQL statement
statement = """
CREATE TABLE IF NOT EXISTS insta_db.final_parquet
WITH (format = 'PARQUET', \
        external_location = 's3://{}/parquet/final_parquet') AS 
with orders as(
select *
, sum(coalesce(days_since_prior_order,0)) over (partition by user_id order by order_number asc) as days
FROM insta_db.orders_parquet
), products as (
select t.*, p.product_name, d.department, a.aisle
from (
SELECT *
FROM insta_db.order_products_prior_parquet
) t
left join insta_db.products_parquet p
on t.product_id = p.product_id
left join insta_db.departments_parquet d
on p.department_id = d.department_id
left join insta_db.aisles_parquet a
on p.aisle_id = a.aisle_id
)
select p.reordered
, p.product_name
, p.department
, p.aisle
, coalesce( days - lag(days) over (partition by o.user_id, p.product_id order by o.order_number) , 0) as days_since_previous_order
from products p
join orders o
on p.order_id = o.order_id
order by p.product_id,
o.order_number ASC
""".format(
    bucket
)

import pandas as pd

pd.read_sql(statement, conn)


In [ ]:
# SQL statement
statement = """
select *
, pow(days_since_previous_order,2) as days_2
, pow(days_since_previous_order,3) as days_3
, pow(days_since_previous_order,4) as days_4
, pow(days_since_previous_order,5) as days_5
, pow(days_since_previous_order,6) as days_6
, pow(days_since_previous_order,7) as days_7
, pow(days_since_previous_order,8) as days_8
from insta_db.final_parquet --limit 200000
"""

import pandas as pd

df = pd.read_sql(statement, conn)
df.head(5)

In [ ]:
# Let's Create Dummies

# product = pd.get_dummies(df['product_name'],drop_first=True)
department = pd.get_dummies(df['department'],drop_first=True)
aisle = pd.get_dummies(df['aisle'],drop_first=True)

# Concatenate new columns with dummies to df
df = pd.concat([df, department, aisle], axis =1)

In [ ]:
print(df.columns)

# Model Logistic Regression / Propensisty Model

In [ ]:
# Get Libraries
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

from matplotlib import pyplot

In [ ]:
sns.countplot(x='reordered', data = df , palette = 'Set1').set(title= 'Number of Repurchased Products')

In [ ]:
Y = pd.DataFrame(df['reordered'])
Y = Y.apply(pd.to_numeric)

X = df.drop(['reordered', 'product_name', 'department', 'aisle', 'days_6','days_7','days_8'], axis = 1)
X = X.apply(pd.to_numeric)

In [ ]:
# Test Train Split Data 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [ ]:
# Build the Model
logmodel = LogisticRegression(class_weight = 'balanced', max_iter =10000000)
logmodel.fit(X_train, y_train.values.ravel())    
predictions_log= logmodel.predict(X_test)

In [ ]:
# Model Evaluation
print(classification_report(y_test,predictions_log))

We can see that our model has a 97% precision rate for not repurchased and a 100% precision rate for repurchase. 

In [ ]:
# Feature Importance
importance = logmodel.coef_.flatten()

In [ ]:
# Barplot of features

pyplot.rcParams["figure.figsize"]=(10,10)
pyplot.barh(X.columns,importance, color = 'g')
pyplot.title("Feature Importance")
pyplot.xlabel("Score")
pyplot.show()


# What will we do with this model

With this propensisty model.  We will use it to score customer product purchase history to determine which of thier past purchases have the highest propensity to be repurchased. This sorted list of possible repurchase products will be used for remarketing purposes for the customer across the website, in emails, and on direct mail flyers. The hopes is to bring the customer back and in so doing, we get the opportunity to cross sell the customer on our other products. 